# Отслеживание окончания трудовых договоров сотрудников ИФИБ
В ИФИБ НИЯУ МИФИ работают сотрудники с определенным сроком, прописанным в трудовом договоре, который необходимо регулярно мониторить, т.к. процесс переоформления / продления / увольнения сотрудников занимает много времени. 

Чтобы избежать накладок и ускорить процесс работы, был разработан алгоритм еженедельного отслеживания окончания трудовых договоров: 
- загрузка файла с актуальным на текущую дату списком сотрудников ИФИБ в Jupiter Notebook
- обработка данных для дальнейшей работы (переименовывание колонок, изменение типа данных, удаление записей о бессрочных трудовых договорах)
- добавление колонки, в которой указано количество оставшихся дней до окончания трудового договора (считая от текущей даты)
- сортировка данных по возрастанию количества оставшихся дней
- определение, есть ли записи, требующие внимания (до окончания трудового договора меньше месяца)
- сохранение этих записей отдельным файлом с указанием текущей даты
- отправление файла по API для дальнейшей работы с отделом кадров

In [371]:
# Грузим либы
import pandas as pd
import numpy as np
import datetime 
import requests
from urllib.parse import urlencode

import warnings
warnings.filterwarnings('ignore')

In [372]:
# Читаем выгрузку
ephib = pd.read_csv('C:/Users/Acer/Python/ифиб-22.01.24.csv')

In [373]:
ephib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   № п/п                      212 non-null    int64  
 1   ФИО                        212 non-null    object 
 2   Шифр                       212 non-null    int64  
 3   E-mail                     183 non-null    object 
 4   Должность                  212 non-null    object 
 5   Категория                  212 non-null    object 
 6   Ставка                     212 non-null    float64
 7   Аудиторная нагрузка        212 non-null    int64  
 8   Руководтсво аспирантами    212 non-null    int64  
 9   Руководство студентами     212 non-null    int64  
 10  Тип                        212 non-null    object 
 11  Подразделение              212 non-null    object 
 12  Дата начала контракта      212 non-null    object 
 13  Дата завершения контракта  212 non-null    object 

In [374]:
# Для удобства переименовываем названия колонок
ephib = ephib.rename(columns={'№ п/п':'№', 'Аудиторная нагрузка':'Аудиторная_нагрузка', 
                            'Руководтсво аспирантами':'Руководство_аспирантами', 'Руководство студентами':'Руководство_студентами',
                            'Дата начала контракта':'Дата_начала_контракта', 'Дата завершения контракта':'Дата_завершения_контракта'})


In [375]:
# Зашифруем личные данные (ФИО и e-mail) (только для анонимности данных в открытом доступе)
ephib['ФИО']    = ephib['ФИО'].apply(lambda x: ''.join(word[0] for word in x.split()))
ephib['E-mail'] = ephib['E-mail'].astype(str).apply(lambda x: x[:3])

In [376]:
# Ищем бессрочные договоры
ephib.query('Дата_завершения_контракта == "-"')

,№,ФИО,Шифр,E-mail,Должность,Категория,Ставка,Аудиторная_нагрузка,Руководство_аспирантами,Руководство_студентами,Тип,Подразделение,Дата_начала_контракта,Дата_завершения_контракта
28,25,ГАА,390667,AAG,Начальник отдела,АУП,1.0,62,0,0,штат,Отдел инновационных технологий в медицине инже...,01.01.2024,-
39,34,ГЕВ,263054,EVG,Заместитель директора,АУП,1.0,162,0,0,штат,Инженерно-физический институт биомедицины НИЯУ...,01.01.2024,-
102,87,ЛНИ,660052,NIL,Заведующий лабораторией,УВП,1.0,16,0,0,штат,Кафедра промышленной фармации (94),01.01.2024,-
128,112,НВВ,521121,VVN,Начальник отдела,АУП,1.0,0,0,0,штат,Организационно-технический отдел инженерно-физ...,01.01.2024,-
169,145,САИ,147662,AIS,Ведущий инженер,УВП,1.0,92,0,0,штат,Кафедра медицинской физики (35),01.12.2008,-
204,176,ШЮВ,25347,YVS,Тьютор,УВП,1.0,219,0,0,штат,Кафедра медицинской физики (35),01.01.2024,-


In [377]:
# Убираем их из df
ephib = ephib.drop(np.where(ephib['Дата_завершения_контракта'] == "-")[0])

In [378]:
# Меняем тип данных для дат
ephib[['Дата_начала_контракта', 'Дата_завершения_контракта']] = ephib[['Дата_начала_контракта', 'Дата_завершения_контракта']].apply(pd.to_datetime, format='%d.%m.%Y')

In [379]:
# Создаем столбец с количеcтвом дней, которое осталось до окончания ТД
ephib['Осталось_дней'] = (ephib['Дата_завершения_контракта'] - pd.Timestamp.now().normalize()).dt.days.astype(int)

In [380]:
# Сортируем по возрастанию числа оставшихся дней
ephib = ephib.sort_values('Осталось_дней', ascending = True)
ephib.head()

,№,ФИО,Шифр,E-mail,Должность,Категория,Ставка,Аудиторная_нагрузка,Руководство_аспирантами,Руководство_студентами,Тип,Подразделение,Дата_начала_контракта,Дата_завершения_контракта,Осталось_дней
89,76,КЕВ,649957,nan,Лаборант,УВП,0.25,0,0,0,штат,Кафедра медицинской физики (35),2023-03-13,2024-01-31,6
51,43,ДОН,672461,OND,Доцент,Поч,1.00,54,0,0,договор,Инженерно-физический институт биомедицины НИЯУ...,2023-11-14,2024-01-31,6
176,151,САИ,670811,AIS,Заведующий отделением,Поч,1.00,34,0,0,договор,Инженерно-физический институт биомедицины НИЯУ...,2023-11-01,2024-01-31,6
70,60,ИДС,667925,DSI,Доцент,ППС,0.10,48,0,0,внешн. совм.,Кафедра ядерной медицины (85),2023-09-22,2024-01-31,6
152,132,СТА,589450,TAS,Доцент,ППС,0.25,93,100,0,внешн. совм.,Отделение биотехнологий офиса образовательных ...,2024-01-01,2024-01-31,6


In [381]:
# Выносим итоговый вердикт и при необходимости сохраняем файл с записями, требующими внимания
if ephib.query('Осталось_дней < 32').shape[0] > 0:
    print('Alarm! Нужно заняться продлением ТД либо увольнением! Следующая проверка ' + str(datetime.date.today() + datetime.timedelta(days=7)))
    alarm_ephib = ephib.query('Осталось_дней < 32')
    alarm_ephib.to_csv('alarm_ephib_{}.csv'.format(datetime.date.today()))
else:
    print('Все в порядке, следующая проверка ' + str(datetime.date.today() + datetime.timedelta(days=7)))

Alarm! Нужно заняться продлением ТД либо увольнением! Следующая проверка 2024-02-01


In [332]:
token = '*****'
chat_id = 724965106  

message = 'Alarm! Нужно заняться продлением ТД либо увольнением! Следующая проверка ' + str(datetime.date.today() + datetime.timedelta(days=7)) 
params = {'chat_id': chat_id, 'text': message}

base_url = f'https://api.telegram.org/bot{token}/'
url_1 = base_url + 'sendMessage?' + urlencode(params)
resp = requests.get(url_1)

filepath = 'C:/Users/Acer/Python/alarm_ephib_{}.csv'.format(datetime.date.today())
url_2 = base_url + 'sendDocument?' + urlencode(params)
files = {'document': open(filepath, 'rb')}
resp = requests.get(url_2, files=files)